In [75]:
import pandas as pd
from glob import glob

In [76]:
datasets = ["agnews", "sst-5", "gsm8k", "copa", "subj"]
models = ["llama", "qwen", "mistral"]
seeds = [42, 43, 44]
dfs = []
for dataset in datasets:
    for model in models:
        for seed in seeds:
            path = glob(
                f"../results/main_results/{dataset}/{model}/CAPO/seed{seed}/*/*/step_results_eval.csv"
            )[0]
            df = pd.read_csv(path)
            dfs.append(
                df.assign(
                    dataset=dataset,
                    model=model,
                    seed=seed,
                )
            )

df = pd.concat(dfs, ignore_index=True)

In [79]:
df["len_instr"] = df["prompt"].str.split("Input:").str[0].apply(lambda x: len(x.split()))
df["len_few_shot"] = df["prompt"].str.split("Input:").str[1].apply(lambda x: len(x.split()))

df["frac_fs"] = df["len_few_shot"] / (df["len_instr"] + df["len_few_shot"])

In [80]:
df = df.sort_values("score", ascending=False)
df = df.groupby(["dataset", "model", "seed"]).last()
df = df.groupby(["dataset", "model"]).mean(numeric_only=True)

In [82]:
df[["len_instr", "len_few_shot", "frac_fs"]]

len_instr  len_few_shot   frac_fs
dataset model                                     
agnews  llama    42.333333     19.666667  0.298037
        mistral  28.000000     42.333333  0.595182
        qwen     31.666667     31.333333  0.387821
copa    llama    33.666667     17.000000  0.277500
        mistral  27.666667     15.333333  0.290966
        qwen     32.666667     14.000000  0.270872
gsm8k   llama    36.000000     75.666667  0.687609
        mistral  26.333333     78.000000  0.731292
        qwen     28.666667     59.000000  0.473391
sst-5   llama    31.666667     16.333333  0.348427
        mistral  25.333333     12.666667  0.204301
        qwen     28.333333     11.333333  0.243379
subj    llama    24.000000      9.333333  0.133333
        mistral  33.000000     19.000000  0.315278
        qwen     31.000000     15.000000  0.251507

In [ ]:
df["frac_downstream_llm"].mean()

KeyError: 'frac_downstream_llm'

In [ ]:
df = df.reset_index()
df.groupby("dataset").mean(numeric_only=True)["frac_downstream_llm"]

dataset
agnews    0.985458
copa      0.871082
gsm8k     0.996258
sst-5     0.990032
subj      0.987852
Name: frac_downstream_llm, dtype: float64

In [ ]:
df.groupby("model").mean(numeric_only=True)["frac_downstream_llm"]

model
llama      0.986693
mistral    0.940054
qwen       0.971662
Name: frac_downstream_llm, dtype: float64